In [3]:
import pandas as pd
import io
import numpy as np

def analyze_improvement_regression(csv_data_string):
    """
    Analyzes a CSV string to calculate improvement and regression percentages
    between iteration 0 and iteration 4 for different experiment types and seeds.

    Args:
        csv_data_string: A string containing the CSV data.

    Returns:
        A string representation of the results table.
    """
    try:
        df = pd.read_csv(io.StringIO(csv_data_string))
    except Exception as e:
        return f"Error reading CSV data: {e}"

    # Ensure required columns exist
    required_cols = ['iteration_0_correct', 'iteration_4_correct', 'seed', 'experiment_type']
    for col in required_cols:
        if col not in df.columns:
            return f"Error: Missing required column '{col}' in the CSV."

    # Convert correctness columns to numeric, coercing errors to NaN
    df['iteration_0_correct'] = pd.to_numeric(df['iteration_0_correct'], errors='coerce')
    df['iteration_4_correct'] = pd.to_numeric(df['iteration_4_correct'], errors='coerce')
    print(len(df))
    # Drop rows where essential correctness data is missing (NaN)
    df.dropna(subset=['iteration_0_correct', 'iteration_4_correct'], inplace=True)
    print(len(df))

    # Filter for rows where correctness columns are only 0 or 1
    # This ensures that only valid binary correctness values are used.
    df = df[df['iteration_0_correct'].isin([0, 1])]
    df = df[df['iteration_4_correct'].isin([0, 1])]

    # Ensure correctness columns are integer type after handling NaNs and filtering
    df['iteration_0_correct'] = df['iteration_0_correct'].astype(int)
    df['iteration_4_correct'] = df['iteration_4_correct'].astype(int)
    print(len(df))

    # Define flags for improved/regressed instances
    df['improved_instance'] = ((df['iteration_0_correct'] == 0) & (df['iteration_4_correct'] == 1)).astype(int)
    df['regressed_instance'] = ((df['iteration_0_correct'] == 1) & (df['iteration_4_correct'] == 0)).astype(int)

    # Calculate counts at the seed level for percentage calculation
    seed_level_calcs = df.groupby(['experiment_type', 'seed']).apply(lambda g: pd.Series({
        'num_improved': g['improved_instance'].sum(),
        'num_regressed': g['regressed_instance'].sum(),
        'total_initially_incorrect': (g['iteration_0_correct'] == 0).sum(),
        'total_initially_correct': (g['iteration_0_correct'] == 1).sum()
    })).reset_index()

    # Calculate percentages at the seed level
    seed_level_calcs['percent_improved'] = seed_level_calcs.apply(
        lambda row: (row['num_improved'] / row['total_initially_incorrect'] * 100)
        if row['total_initially_incorrect'] > 0 else np.nan, # np.nan if denominator is 0
        axis=1
    )
    seed_level_calcs['percent_regressed'] = seed_level_calcs.apply(
        lambda row: (row['num_regressed'] / row['total_initially_correct'] * 100)
        if row['total_initially_correct'] > 0 else np.nan, # np.nan if denominator is 0
        axis=1
    )
    
    # Custom SEM function: std_dev / sqrt(n)
    def sem_fn(x):
        n = x.count() # count() correctly ignores NaNs
        if n == 0:
            return np.nan
        std_dev = x.std() # std() correctly ignores NaNs
        return std_dev / (n**0.5) if n > 0 else np.nan

    # Aggregate percentages to get mean and SEM for each experiment_type
    experiment_stats_percent = seed_level_calcs.groupby('experiment_type').agg(
        improved_percent_mean=('percent_improved', 'mean'),
        improved_percent_sem=('percent_improved', sem_fn),
        regressed_percent_mean=('percent_regressed', 'mean'),
        regressed_percent_sem=('percent_regressed', sem_fn)
    ) # experiment_type becomes the index

    # Format the output table
    final_table_data = {}
    if experiment_stats_percent.empty:
        return "No data to aggregate after percentage calculation. Check input data and filters."
        
    for exp_type in experiment_stats_percent.index:
        imp_mean = experiment_stats_percent.loc[exp_type, 'improved_percent_mean']
        imp_sem = experiment_stats_percent.loc[exp_type, 'improved_percent_sem']
        reg_mean = experiment_stats_percent.loc[exp_type, 'regressed_percent_mean']
        reg_sem = experiment_stats_percent.loc[exp_type, 'regressed_percent_sem']

        final_table_data[exp_type] = {
            'Improved (%)': f"{imp_mean:.2f} \u00B1 {imp_sem:.2f}",
            'Regressed (%)': f"{reg_mean:.2f} \u00B1 {reg_sem:.2f}"
        }
    
    if not final_table_data: # Should be caught by experiment_stats_percent.empty already
        return "No results to display. This might be due to empty input or no valid data after filtering."

    result_df = pd.DataFrame(final_table_data)
    
    if result_df.empty:
         return "Resulting DataFrame is empty after processing."

    output_string = "Analysis of Improvement and Regression Percentages (Iteration 4 vs. Iteration 0):\n"
    output_string += result_df.to_string()
    
    return output_string, seed_level_calcs

# To use this function:
# 1. Get your CSV data into a string variable (e.g., by reading the file).
csv_file_path = '/Users/justinshen/Documents/Code/cocolab/reasoning-distillation/paper_results/aggregated_iteration_grading_results_with_year.csv'
with open(csv_file_path, 'r') as f:
    csv_data_content = f.read()

# 2. Call the function:
analysis_result, seed_level_calcs = analyze_improvement_regression(csv_data_content)
print(analysis_result)


1680
1680
1680
Analysis of Improvement and Regression Percentages (Iteration 4 vs. Iteration 0):
                answer_only  backtracking      base_sum        firstk         lastk     postThink         wait
Improved (%)   30.26 ± 6.76  31.81 ± 3.49  27.88 ± 2.90  21.85 ± 3.64  23.19 ± 2.71  30.11 ± 4.00  2.00 ± 2.00
Regressed (%)   8.86 ± 3.55   0.66 ± 0.66   4.03 ± 0.85  24.99 ± 3.70   3.23 ± 2.50   2.57 ± 1.86  0.60 ± 0.60


In [4]:
seed_level_calcs

,experiment_type,seed,num_improved,num_regressed,total_initially_incorrect,total_initially_correct,percent_improved,percent_regressed
0,answer_only,seed0,7,2,25,35,28.000000,5.714286
1,answer_only,seed1,5,1,24,36,20.833333,2.777778
2,answer_only,seed2,11,3,22,38,50.000000,7.894737
3,answer_only,seed3,4,8,18,42,22.222222,19.047619
4,backtracking,seed0,7,0,25,35,28.000000,0.000000
5,backtracking,seed1,6,0,24,36,25.000000,0.000000
6,backtracking,seed2,9,1,22,38,40.909091,2.631579
7,backtracking,seed3,6,0,18,42,33.333333,0.000000
8,base_sum,seed0,9,2,25,35,36.000000,5.714286
9,base_sum,seed1,6,1,24,36,25.000000,2.777778
